In [1]:
# Load libraries
import numpy as np
from cobra.io import read_sbml_model, write_sbml_model
from cobra.util import create_stoichiometric_matrix
from cobra import Reaction, Metabolite
from cobra.core.gene import GPR


#From Marks report
from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield
from cameo.strain_design.heuristic.evolutionary_based import CofactorSwapOptimization
#from cobra.io import read_sbml_model
#model = read_sbml_model('models/yeast_lyco.xml')

In [2]:
from cameo.strain_design.heuristic.evolutionary.optimization import CofactorSwapOptimization
from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield
from cameo.strain_design.heuristic.evolutionary.objective_functions import biomass_product_coupled_yield
from cameo.util import TimeMachine
from cameo.flux_analysis.analysis import flux_variability_analysis as fva

In [3]:
# Loading model
infilename = '../../models/iMM904_progesterone_fixed.xml'
print(f"Loading {infilename}")
model = read_sbml_model(infilename)
#model2 = load_model(infilename)

Loading ../../models/iMM904_progesterone_fixed.xml


In [4]:
# This information is needed to make the model produce progesterone. Otherwise, no progesterone is produced. 

model.add_boundary(model.metabolites.get_by_id('progesterone_c'), type='demand')

Reaction identifier,DM_progesterone_c
Name,progesterone demand
Memory address,0x07f134cd2bc10
Stoichiometry,progesterone_c --> progesterone -->
GPR,
Lower bound,0
Upper bound,1000.0


In [7]:
# What is this: optimize the model. How?
Fluxes=model.optimize()
Fluxes

,fluxes,reduced_costs
CITtcp,0.000000e+00,-0.000000e+00
13BGH,0.000000e+00,1.783417e-18
13BGHe,0.000000e+00,-1.925930e-34
13GS,-1.333313e-16,-4.622232e-33
16GS,0.000000e+00,3.566833e-18
...,...,...
R07215,1.428571e-01,6.626363e-17
R04804,1.428571e-01,-5.551115e-16
R_methylpentanal_t,1.428571e-01,-5.986817e-17
SK_methylpentanal_e,1.428571e-01,-1.797603e-16


In [8]:
#Choose the reaction that produced progesterone as the objective.
model.objective = model.reactions.R02216

In [9]:
#Calculate the product yield 


py = product_yield(model.reactions.R02216, model.reactions.ECYP11A1)
optswap = CofactorSwapOptimization(model=model, objective_function=py)

In [10]:
swap_optimization = CofactorSwapOptimization(model=model, cofactor_id_swaps=(["nad_c","nadh_c"],["fad_c","fadh2_m"]),objective_function=py)
# VIRKER!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

#swap_optimization = CofactorSwapOptimization(model=model,objective_function=py)
swap_optimization

In [11]:
solution=swap_optimization.run(max_size=10)

Starting optimization at Thu, 17 Nov 2022 18:25:39


/home/trinebera/miniconda3/envs/thedream/lib/python3.9/site-packages/cameo/strain_design/heuristic/evolutionary/objective_functions.py:353: RuntimeWarning: invalid value encountered in double_scalars
  return round(product_flux / substrate_flux, config.ndecimals)
/home/trinebera/miniconda3/envs/thedream/lib/python3.9/site-packages/cameo/strain_design/heuristic/evolutionary/objective_functions.py:353: RuntimeWarning: invalid value encountered in double_scalars
  return round(product_flux / substrate_flux, config.ndecimals)
/home/trinebera/miniconda3/envs/thedream/lib/python3.9/site-packages/cameo/strain_design/heuristic/evolutionary/objective_functions.py:353: RuntimeWarning: invalid value encountered in double_scalars
  return round(product_flux / substrate_flux, config.ndecimals)
/home/trinebera/miniconda3/envs/thedream/lib/python3.9/site-packages/cameo/strain_design/heuristic/evolutionary/objective_functions.py:353: RuntimeWarning: invalid value encountered in double_scalars
  return

HBox()

Finished after 00:00:52


/home/trinebera/miniconda3/envs/thedream/lib/python3.9/site-packages/cameo/strain_design/heuristic/evolutionary/objective_functions.py:353: RuntimeWarning: invalid value encountered in double_scalars
  return round(product_flux / substrate_flux, config.ndecimals)
/home/trinebera/miniconda3/envs/thedream/lib/python3.9/site-packages/cameo/strain_design/heuristic/evolutionary/objective_functions.py:353: RuntimeWarning: invalid value encountered in double_scalars
  return round(product_flux / substrate_flux, config.ndecimals)
/home/trinebera/miniconda3/envs/thedream/lib/python3.9/site-packages/cameo/strain_design/heuristic/evolutionary/objective_functions.py:353: RuntimeWarning: invalid value encountered in double_scalars
  return round(product_flux / substrate_flux, config.ndecimals)
/home/trinebera/miniconda3/envs/thedream/lib/python3.9/site-packages/cameo/strain_design/heuristic/evolutionary/objective_functions.py:353: RuntimeWarning: invalid value encountered in double_scalars
  return

In [12]:
solution

,index,targets,fitness
0,0,"(MDH, R02216)",0.999999
1,1,"(R02216, HSDxi)",0.999999
2,2,"(R02216, LNS14DMx)",0.999999
3,3,"(LNS14DMx, R02216)",0.999999
4,4,"(GLUSx, R02216)",0.999999
5,5,"(ALCD23xi, R02216)",0.999999
6,7,"(R02216, MDH)",0.999999
7,8,"(GAPD,)",0.875007
8,11,"(ALCD26xi,)",0.874999
9,12,"(G3PD1ir,)",0.874999


In [13]:
reactions=solution.data_frame.targets
reactions

0         (MDH, R02216)
1       (R02216, HSDxi)
2    (R02216, LNS14DMx)
3    (LNS14DMx, R02216)
4       (GLUSx, R02216)
5    (ALCD23xi, R02216)
6         (R02216, MDH)
7               (GAPD,)
8           (ALCD26xi,)
9            (G3PD1ir,)
Name: targets, dtype: object

In [14]:
for t in reactions:
    print("Reactions:")
    for id in t:
        r=model.reactions.get_by_id(id)
        #print(r.name, id+",","Fitness:", solution.data_frame.fitness[id][10])
        s=""
        for rea in range(len(r.reactants)):
            if rea==len(r.reactants)-1:
                s+=r.reactants[rea].name+" ==> "
            else:
                s+=r.reactants[rea].name+" + "
        for p in range(len(r.products)):
            if p==len(r.products)-1:
                s+=r.products[p].name
            else:
                s+=r.products[p].name+" + "
        print(s)
        print()

Reactions:
L-Malate + Nicotinamide adenine dinucleotide ==> H+ + Oxaloacetate + Nicotinamide adenine dinucleotide - reduced

pregnenolone + Nicotinamide adenine dinucleotide ==> progesterone + H+ + Nicotinamide adenine dinucleotide - reduced

Reactions:
pregnenolone + Nicotinamide adenine dinucleotide ==> progesterone + H+ + Nicotinamide adenine dinucleotide - reduced

L-Aspartate 4-semialdehyde + H+ + Nicotinamide adenine dinucleotide - reduced ==> L-Homoserine + Nicotinamide adenine dinucleotide

Reactions:
pregnenolone + Nicotinamide adenine dinucleotide ==> progesterone + H+ + Nicotinamide adenine dinucleotide - reduced

H+ + Lanosterol C30H50O + O2 O2 + Nicotinamide adenine dinucleotide - reduced ==> 4 4 dimethylcholesta 8 14 24 trienol C29H46O + Formate + H2O H2O + Nicotinamide adenine dinucleotide

Reactions:
H+ + Lanosterol C30H50O + O2 O2 + Nicotinamide adenine dinucleotide - reduced ==> 4 4 dimethylcholesta 8 14 24 trienol C29H46O + Formate + H2O H2O + Nicotinamide adenine di